In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as skl
from sklearn.model_selection import train_test_split
import tensorflow as tf
import seaborn as sns

In [ ]:
df = pd.read_csv('E:/NASA STAR CLASSIFICATION DATASET.csv')
df['Type'] = df['Type'].astype(str)
df

In [ ]:
df['Color'] = df['Color'].replace(['Blue White','Blue white','Blue-white'],'Blue-White')
df['Color'] = df['Color'].replace(['yellow-white','White-Yellow'],'Yellowish White')
df['Color'] = df['Color'].replace(['yellowish'],'Yellowish')
df['Color'] = df['Color'].replace(['Whitish','white'],'White')

sns.catplot(x='Temperature',y='Color',data= df)

In [ ]:
dict_type = {'0':'Red Dwarf','1': 'Brown Dwarf','2': 'White Dwarf','3': 'Main Sequence','4': 'Supergiants','5': 'Hypergiants'}
df['Type'] = df['Type'].replace(dict_type.keys(),dict_type.values())
df

In [ ]:
df_enc = pd.get_dummies(data=df.drop('Type',axis=1), columns=['Color','Spectral_Class'],drop_first= True)

In [ ]:
df_enc.shape

In [ ]:
X = df_enc
y = pd.DataFrame(df['Type'],columns=['Type'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size = 0.1,random_state = 20)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
X_train_mm = pd.DataFrame(mm.fit_transform(X_train),columns=['Temperature', 'L', 'R', 'A_M', 'Color_Blue-White', 'Color_Orange',
       'Color_Orange-Red', 'Color_Pale yellow orange', 'Color_Red',
       'Color_White', 'Color_Yellowish', 'Color_Yellowish White',
       'Spectral_Class_B', 'Spectral_Class_F', 'Spectral_Class_G',
       'Spectral_Class_K', 'Spectral_Class_M', 'Spectral_Class_O'])
X_test_mm = pd.DataFrame(mm.transform(X_test),columns=['Temperature', 'L', 'R', 'A_M', 'Color_Blue-White', 'Color_Orange',
       'Color_Orange-Red', 'Color_Pale yellow orange', 'Color_Red',
       'Color_White', 'Color_Yellowish', 'Color_Yellowish White',
       'Spectral_Class_B', 'Spectral_Class_F', 'Spectral_Class_G',
       'Spectral_Class_K', 'Spectral_Class_M', 'Spectral_Class_O'])


In [ ]:
X_train_mm

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lr = LogisticRegression()
model = lr.fit(X_train_mm,y_train)
predictions = model.predict(X_test_mm)
model_score = model.score(X_test_mm,y_test)
print("THE ACCURACY OF LOGISTIC REGRESSION CLASSIFIER IS",model_score)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
model_rf = rf.fit(X_train_mm,y_train)
predictions = model_rf.predict(X_test_mm)
model_score_rf = model_rf.score(X_test_mm,y_test)
print("THE ACCURACY OF RANDOM FOREST CLASSIFIER IS",model_score_rf)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1,metric='euclidean',weights='uniform')
model_knn = knn.fit(X_train_mm,y_train)
model_knn_score = knn.score(X_test_mm,y_test)
model_knn_score
model_knn_predictions = knn.predict(X_test_mm)
print("THE TEST ACCURACY OF KNN CLASSIFIER IS",model_knn_score)

In [ ]:
from tensorflow import keras
y_train = pd.get_dummies(data=y_train,columns =['Type'])
y_test = pd.get_dummies(data=y_test,columns =['Type'])

In [ ]:
model_nn = keras.Sequential([keras.layers.Dense(500,kernel_initializer = 'he_uniform',input_shape=(18,),activation = 'relu'),
                            keras.layers.Dense(720,kernel_initializer = 'he_uniform',activation = 'relu'),
                            keras.layers.Dense(800,kernel_initializer = 'he_uniform',activation = 'relu'),
                            keras.layers.Dense(6,kernel_initializer = 'glorot_uniform',activation = 'softmax'),
                            ])
from tensorflow.keras.callbacks import  EarlyStopping
es = EarlyStopping(patience=3,restore_best_weights= True)
model_nn.compile(loss = "categorical_crossentropy",metrics = ['accuracy'],optimizer = 'sgd')
history = model_nn.fit(X_train_mm,y_train,validation_data = (X_test_mm,y_test),epochs = 30,callbacks = es)

In [ ]:
test_loss,test_acc= model_nn.evaluate(X_test_mm,y_test)
print("test_accuracy:",test_acc)


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Performance')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training_loss', 'validation_loss'], loc='upper right')
plt.show()

We can see that the model performs really well. There is no overfitting.

THE TEST ACCURACY OF LOGISTIC REGRESSION IS 95.83%

THE TEST ACCURACY OF RANDOM FOREST CLASSIFIER IS 100%

THE TEST ACCURACY OF KNN CLASSIFIER IS 100%

THE TEST ACCURACY OF ARTIFICIAL NEURAL NETWORK IS 95.8%